# Домашнее задание к вебинару 8
Задание  
Взять берем отызывы за лето  
Выяснить какая архитектура больше подходит для нашей задачи 
1. построить свёрточные архитектуры 
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [153]:
import pandas as pd
from sklearn.model_selection import train_test_split

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

from keras.losses import SparseCategoricalCrossentropy
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, MaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping 
from tensorflow.keras.utils import to_categorical

**Загрузка и предобработка данных**.  
Заранее перевела данные в формат csv

In [6]:
df = pd.read_csv('/content/отзывы за лето.csv')
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.head(2)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14


In [7]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((13841, 3), (6818, 3))

**Предобработка данных**

In [8]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [9]:
text_corpus_train = df_train['Content'].values
text_corpus_test = df_test['Content'].values

In [30]:
len(text_corpus_train) == df_train['Content'].shape[0]

True

In [10]:
len(text_corpus_train), text_corpus_train 

(13841,
 array(['наконецтый исправить чушь снеоргинальный прошивка приложение удобно пользоваться',
        'удобно использование', 'отлично', ...,
        'мочь скачать ошибка номер 24', 'сбербанк', 'целое отлично'],
       dtype=object))

In [23]:
# Определяем токенайзер
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
# Обучаем токенайзер на тренировочном наборе данных - > получаем готовый словарь
tokenizer.fit_on_texts(text_corpus_train)

# Выводим словарь
# tokenizer.index_word

In [24]:
# На основе полученного словаря токены переводятся в их id и сформировываем последовательности sequences

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

In [27]:
# всего слов в словаре
word_count = len(tokenizer.index_word) + 1

# находим максимальную длину будущей последовательности в тренировочном наборе отзывов
training_length = max([len(i.split()) for i in text_corpus_train])

word_count, training_length 

(10293, 113)

In [83]:
X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

In [32]:
X_train[5]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0, 127,   1], dtype=int32)

In [33]:
# Преобразование значений y
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [37]:
train_enc_labels

array([4, 4, 4, ..., 0, 4, 4])

In [42]:
num_classes = df_train['Rating'].nunique()
y_train = to_categorical(train_enc_labels, num_classes)
y_test = to_categorical(test_enc_labels, num_classes)

In [48]:
y_train.shape, y_train[3]

((13841, 5), array([0., 0., 0., 0., 1.], dtype=float32))

### 1. CNN (Сверточные нейронные сети)

In [200]:
model = Sequential()
model.add(Embedding(input_dim=word_count, output_dim=256, input_length=training_length)) 
                    #inputdim -- размер словаря, outputdim -- длина вектора, input_length -- длина входной последовательности
                    #на вход: (батч, input_lenght), на выходе: (батч, input_lenght, output_dim)

model.add(Conv1D(128, 3, padding = 'same'))
                   #64 -- количество 1D-фильтров, длина фильтра -- 3
                   #на выходе (количество объектов, 128 - длина нового эмбединга)

model.add(MaxPool1D(2))  
model.add(Activation("relu"))


model.add(Conv1D(256, 3, padding = 'same'))
model.add(MaxPool1D(2))
model.add(Activation("relu"))
                    #применяем функцию активации к выходу предыдущего слоя

model.add(GlobalMaxPool1D())
                    # в каждой свертке для каждого элемента эмбединга оставляем максимум - получаем один 128-м эмбеддинг

model.add(Dense(128))
               #128-количество выходов
model.add(Activation("relu"))
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(num_classes))
                #num_classes = 5 -- количество выходов
model.add(Activation('softmax'))
                #преобразуем вектор в раcgределение вероятностей
  
model.summary()

Model: "sequential_63"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_46 (Embedding)    (None, 113, 256)          2635008   
                                                                 
 conv1d_55 (Conv1D)          (None, 113, 128)          98432     
                                                                 
 max_pooling1d_31 (MaxPoolin  (None, 56, 128)          0         
 g1D)                                                            
                                                                 
 activation_81 (Activation)  (None, 56, 128)           0         
                                                                 
 conv1d_56 (Conv1D)          (None, 56, 256)           98560     
                                                                 
 max_pooling1d_32 (MaxPoolin  (None, 28, 256)          0         
 g1D)                                                

In [201]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [202]:
early_stopping=EarlyStopping(monitor='val_loss', patience = 1, mode = 'min')  


history = model.fit(X_train, y_train,
                    batch_size=256,
                    epochs=10,
                    verbose=1,
                    validation_data = (X_test, y_test),
                    callbacks=[early_stopping]
                    )

Epoch 1/10
55/55 [==============================] - 42s 742ms/step - loss: 0.9172 - accuracy: 0.7155 - val_loss: 0.6772 - val_accuracy: 0.7640
Epoch 2/10
55/55 [==============================] - 40s 735ms/step - loss: 0.6346 - accuracy: 0.7752 - val_loss: 0.6491 - val_accuracy: 0.7738
Epoch 3/10
55/55 [==============================] - 39s 716ms/step - loss: 0.5378 - accuracy: 0.8075 - val_loss: 0.6842 - val_accuracy: 0.7589


In [203]:
cnn_score = model.evaluate(X_test, y_test, batch_size=256, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

27/27 [==============================] - 4s 144ms/step - loss: 0.6842 - accuracy: 0.7589


Test score: 0.2265089601278305
Test accuracy: 0.7692871689796448


### 2. RNN (Рекуррентные нейронные сети)
#### 2.1 Simple RNN

In [204]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))  #64 -- количество ячеек, размер выходного пространства - количество скрытых состояний на момент последнего эл-тв послед-сти
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


model.summary()

Model: "sequential_64"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_47 (Embedding)    (None, 113, 30)           308790    
                                                                 
 masking_31 (Masking)        (None, 113, 30)           0         
                                                                 
 simple_rnn_13 (SimpleRNN)   (None, 64)                6080      
                                                                 
 dense_108 (Dense)           (None, 64)                4160      
                                                                 
 dropout_28 (Dropout)        (None, 64)                0         
                                                                 
 dense_109 (Dense)           (None, 5)                 325       
                                                                 
Total params: 319,355
Trainable params: 319,355
Non-t

In [205]:
%%time 

model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_data = (X_test, y_test),
                    callbacks = [early_stopping]
                    )

Epoch 1/10
28/28 [==============================] - 8s 250ms/step - loss: 0.4566 - accuracy: 0.5935 - val_loss: 0.3211 - val_accuracy: 0.7125
Epoch 2/10
28/28 [==============================] - 6s 211ms/step - loss: 0.3531 - accuracy: 0.6984 - val_loss: 0.3129 - val_accuracy: 0.7125
Epoch 3/10
28/28 [==============================] - 6s 206ms/step - loss: 0.3174 - accuracy: 0.7038 - val_loss: 0.2679 - val_accuracy: 0.7169
Epoch 4/10
28/28 [==============================] - 6s 207ms/step - loss: 0.2714 - accuracy: 0.7291 - val_loss: 0.2440 - val_accuracy: 0.7452
Epoch 5/10
28/28 [==============================] - 6s 209ms/step - loss: 0.2484 - accuracy: 0.7549 - val_loss: 0.2348 - val_accuracy: 0.7561
Epoch 6/10
28/28 [==============================] - 6s 208ms/step - loss: 0.2319 - accuracy: 0.7710 - val_loss: 0.2307 - val_accuracy: 0.7617
Epoch 7/10
28/28 [==============================] - 6s 209ms/step - loss: 0.2141 - accuracy: 0.7857 - val_loss: 0.2291 - val_accuracy: 0.7646
Epoch 

In [206]:
srnn_score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 1s 60ms/step - loss: 0.2372 - accuracy: 0.7636


Test score: 0.2265089601278305
Test accuracy: 0.7692871689796448


#### 2.2 LSTM (Long short term memory)

In [207]:
%%time 

model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(64))  #64 -- количество ячеек, размер выходного пространства - количество скрытых состояний на момент последнего эл-тв послед-сти
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_data = (X_test, y_test),
                    callbacks = [early_stopping]
                    )

Epoch 1/10
28/28 [==============================] - 22s 637ms/step - loss: 0.5351 - accuracy: 0.6505 - val_loss: 0.3201 - val_accuracy: 0.7125
Epoch 2/10
28/28 [==============================] - 16s 583ms/step - loss: 0.3627 - accuracy: 0.6875 - val_loss: 0.3049 - val_accuracy: 0.7125
Epoch 3/10
28/28 [==============================] - 16s 589ms/step - loss: 0.3130 - accuracy: 0.7052 - val_loss: 0.2593 - val_accuracy: 0.7351
Epoch 4/10
28/28 [==============================] - 16s 588ms/step - loss: 0.2715 - accuracy: 0.7374 - val_loss: 0.2387 - val_accuracy: 0.7501
Epoch 5/10
28/28 [==============================] - 17s 592ms/step - loss: 0.2475 - accuracy: 0.7560 - val_loss: 0.2294 - val_accuracy: 0.7581
Epoch 6/10
28/28 [==============================] - 16s 591ms/step - loss: 0.2321 - accuracy: 0.7680 - val_loss: 0.2286 - val_accuracy: 0.7630
Epoch 7/10
28/28 [==============================] - 18s 627ms/step - loss: 0.2200 - accuracy: 0.7791 - val_loss: 0.2279 - val_accuracy: 0.7655

In [208]:
lstm_score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 2s 170ms/step - loss: 0.2324 - accuracy: 0.7659


Test score: 0.2265089601278305
Test accuracy: 0.7692871689796448


#### 2.3 GRU (Gated recurrent unit)

In [209]:
%%time 

model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(64))  #64 -- количество ячеек, размер выходного пространства - количество скрытых состояний на момент последнего эл-тв послед-сти
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_data = (X_test, y_test),
                    callbacks = [early_stopping]
                    )

Epoch 1/10
28/28 [==============================] - 22s 633ms/step - loss: 0.5403 - accuracy: 0.6110 - val_loss: 0.3196 - val_accuracy: 0.7125
Epoch 2/10
28/28 [==============================] - 16s 584ms/step - loss: 0.3607 - accuracy: 0.6947 - val_loss: 0.3133 - val_accuracy: 0.7125
Epoch 3/10
28/28 [==============================] - 16s 588ms/step - loss: 0.3268 - accuracy: 0.7013 - val_loss: 0.2774 - val_accuracy: 0.7130
Epoch 4/10
28/28 [==============================] - 16s 583ms/step - loss: 0.2938 - accuracy: 0.7189 - val_loss: 0.2542 - val_accuracy: 0.7310
Epoch 5/10
28/28 [==============================] - 16s 588ms/step - loss: 0.2633 - accuracy: 0.7454 - val_loss: 0.2354 - val_accuracy: 0.7545
Epoch 6/10
28/28 [==============================] - 16s 586ms/step - loss: 0.2433 - accuracy: 0.7588 - val_loss: 0.2341 - val_accuracy: 0.7551
Epoch 7/10
28/28 [==============================] - 16s 590ms/step - loss: 0.2332 - accuracy: 0.7674 - val_loss: 0.2284 - val_accuracy: 0.7590

In [210]:
gru_score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 2s 169ms/step - loss: 0.2271 - accuracy: 0.7669


Test score: 0.2265089601278305
Test accuracy: 0.7692871689796448


**Вывод:** из всех рекурретных нейронных сетей LSTM дал наилучший результат, но он обучался дольше всех.

### 3. RNN + CNN
3.1 CNN -> RNN

In [211]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(LSTM(64,recurrent_dropout=0.2)) 
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.summary()

Model: "sequential_67"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_50 (Embedding)    (None, 113, 30)           308790    
                                                                 
 masking_34 (Masking)        (None, 113, 30)           0         
                                                                 
 conv1d_57 (Conv1D)          (None, 111, 128)          11648     
                                                                 
 activation_86 (Activation)  (None, 111, 128)          0         
                                                                 
 lstm_21 (LSTM)              (None, 64)                49408     
                                                                 
 dense_114 (Dense)           (None, 32)                2080      
                                                                 
 dropout_31 (Dropout)        (None, 32)              

In [212]:
model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_data = (X_test, y_test),
                    callbacks = [early_stopping]
                    )

Epoch 1/10
28/28 [==============================] - 31s 1s/step - loss: 0.5349 - accuracy: 0.6051 - val_loss: 0.3176 - val_accuracy: 0.7125
Epoch 2/10
28/28 [==============================] - 27s 962ms/step - loss: 0.3754 - accuracy: 0.6669 - val_loss: 0.2765 - val_accuracy: 0.7125
Epoch 3/10
28/28 [==============================] - 27s 962ms/step - loss: 0.3201 - accuracy: 0.6989 - val_loss: 0.2475 - val_accuracy: 0.7317
Epoch 4/10
28/28 [==============================] - 27s 970ms/step - loss: 0.2860 - accuracy: 0.7239 - val_loss: 0.2342 - val_accuracy: 0.7536
Epoch 5/10
28/28 [==============================] - 27s 968ms/step - loss: 0.2632 - accuracy: 0.7405 - val_loss: 0.2321 - val_accuracy: 0.7599
Epoch 6/10
28/28 [==============================] - 27s 973ms/step - loss: 0.2450 - accuracy: 0.7594 - val_loss: 0.2306 - val_accuracy: 0.7633
Epoch 7/10
28/28 [==============================] - 27s 972ms/step - loss: 0.2316 - accuracy: 0.7687 - val_loss: 0.2351 - val_accuracy: 0.7655


In [213]:
cnn_rnn_score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 3s 212ms/step - loss: 0.2351 - accuracy: 0.7655


Test score: 0.2265089601278305
Test accuracy: 0.7692871689796448


3.2 RNN -> CNN

In [214]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(64,recurrent_dropout=0.2, return_sequences=True))
  #return_sequences = True с- возвращаем последовательность скрытых состояний для каждого токена
model.add(Conv1D(64, 3, padding = 'same', activation = 'relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.summary()

Model: "sequential_68"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_51 (Embedding)    (None, 113, 30)           308790    
                                                                 
 masking_35 (Masking)        (None, 113, 30)           0         
                                                                 
 lstm_22 (LSTM)              (None, 113, 64)           24320     
                                                                 
 conv1d_58 (Conv1D)          (None, 113, 64)           12352     
                                                                 
 global_max_pooling1d_27 (Gl  (None, 64)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_116 (Dense)           (None, 32)                2080      
                                                     

In [215]:
model.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_data = (X_test, y_test),
                    callbacks = [early_stopping]
                    )

Epoch 1/10
28/28 [==============================] - 26s 804ms/step - loss: 0.5404 - accuracy: 0.5515 - val_loss: 0.3806 - val_accuracy: 0.7125
Epoch 2/10
28/28 [==============================] - 23s 830ms/step - loss: 0.4207 - accuracy: 0.6481 - val_loss: 0.3412 - val_accuracy: 0.7125
Epoch 3/10
28/28 [==============================] - 22s 792ms/step - loss: 0.3978 - accuracy: 0.6729 - val_loss: 0.3327 - val_accuracy: 0.7125
Epoch 4/10
28/28 [==============================] - 22s 794ms/step - loss: 0.3813 - accuracy: 0.6912 - val_loss: 0.3192 - val_accuracy: 0.7125
Epoch 5/10
28/28 [==============================] - 22s 790ms/step - loss: 0.3498 - accuracy: 0.7002 - val_loss: 0.2613 - val_accuracy: 0.7452
Epoch 6/10
28/28 [==============================] - 22s 797ms/step - loss: 0.2913 - accuracy: 0.7273 - val_loss: 0.2376 - val_accuracy: 0.7583
Epoch 7/10
28/28 [==============================] - 22s 795ms/step - loss: 0.2605 - accuracy: 0.7449 - val_loss: 0.2305 - val_accuracy: 0.7622

In [216]:
rnn_cnn_score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 2s 154ms/step - loss: 0.2278 - accuracy: 0.7684


Test score: 0.2265089601278305
Test accuracy: 0.7692871689796448


**Соберем все результаты вместе**

In [218]:
result = pd.DataFrame({'model': ['CNN', 'SimpleRNN', 'LSTM', 'GRU', 'CNN->LSTM', 'LSTM->CNN'], 
                       'accuracy': [cnn_score[1], srnn_score[1], lstm_score[1], gru_score[1], cnn_rnn_score[1], rnn_cnn_score[1]]})
result.sort_values(by='accuracy', ascending=False)

,model,accuracy
5,LSTM->CNN,0.768407
3,GRU,0.766940
2,LSTM,0.765914
4,CNN->LSTM,0.765474
1,SimpleRNN,0.763567
0,CNN,0.758874


Наилучший результат показала модель со слоями LSTM, и затем CNN, немного ей уступили сети со слоем GRU и LSTM.
CNN показала самый слабый результат в работе с текстом.